In [9]:
!pip install transformers
!pip install torch

In [10]:
from transformers import BertForQuestionAnswering
from transformers import BertTokenizer
import torch
import numpy as np


In [11]:
model = BertForQuestionAnswering.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")

Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [12]:
tokenizer_for_bert = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

In [13]:
#max_len is maximum size of passage
def bert_question_answer(question, passage, max_len=500):

    #Tokenize input question and passage
    #Include unique tokens- [CLS] and [SEP]
    input_ids = tokenizer_for_bert.encode(question,passage,max_length = max_len,truncation=True)

    #Getting number of tokens in 1st sentence (question) and 2nd sentence (passage that contains answer)
    sep_index = input_ids.index(102)
    len_question = sep_index + 1
    len_passage = len(input_ids) - len_question

    tokens = tokenizer_for_bert.convert_ids_to_tokens(input_ids)

     #Converting token ids to tokens
    segment_ids = [0]*len_question + [1]*(len_passage)

    #Getting start and end scores for answer
    #Converting input arrays to torch tensors before passing to the model
    start_token_scores = model(torch.tensor([input_ids]), token_type_ids=torch.tensor([segment_ids]))[0]
    end_token_scores = model(torch.tensor([input_ids]), token_type_ids=torch.tensor([segment_ids]))[1]

    #Converting scores tensors to numpy arrays
    start_token_scores = start_token_scores.detach().numpy().flatten()
    end_token_scores = end_token_scores.detach().numpy().flatten()

    #Getting start and end index of answer based on highest scores
    answer_start_index = np.argmax(start_token_scores)
    answer_end_index = np.argmax(end_token_scores)

    start_token_score = np.round(start_token_scores[answer_start_index],2)
    end_token_score = np.round(end_token_scores[answer_end_index],2)

    answer = tokens[answer_start_index]
    for i in range(answer_start_index + 1, answer_end_index + 1):
        if tokens[i][0:2] == "##":
            answer += tokens[i][2:]
        else:
            answer += ' ' + tokens[i]

    # If the answer didn't find in the passage
    if(answer_start_index == 0) or (start_token_score < 0) or (answer == '[SEP]') or (answer_end_index < answer_start_index):
        answer = "Sorry!,I could not find an answer in passage."
    return(answer_start_index,answer_end_index,start_token_score,end_token_score,answer)

#Testing function
bert_question_answer("What is my name","Hello, I am Ishwar. My friend name is Ajay. He is the son of Kristen. I spend most of the time with Ajay.He always call me by my nick name. Ajay call me programmer. Except Ajay, my other friend call me by my original name. Bijay is also my friend.")





(10, 12, 3.89, 4.91, 'ishwar')

In [14]:
passage = "Cricket, England’s national summer sport, which is now played throughout the world, particularly in Australia, India, Pakistan, the West Indies, and the British Isles.Cricket is played with a bat and ball and involves two competing sides (teams) of 11 players. The field is oval with a rectangular area in the middle, known as the pitch, that is 22 yards (20.12 metres) by 10 feet (3.04 metres) wide. Two sets of three sticks, called wickets, are set in the ground at each end of the pitch. Across the top of each wicket lie horizontal pieces called bails. The sides take turns at batting and bowling (pitching); each turn is called an “innings” (always plural). Sides have one or two innings each, depending on the prearranged duration of the match, the object being to score the most runs. The bowlers, delivering the ball with a straight arm, try to break (hit) the wicket with the ball so that the bails fall. This is one of several ways that the batsman is dismissed, or put out. A bowler delivers six balls at one wicket (thus completing an “over”), then a different player from his side bowls six balls to the opposite wicket. The batting side defends its wicket."

print(f"Length of the passage: {len(passage.split())} words")

question1 = "who play cricket"
_, _, _, _,ans = bert_question_answer(question1,passage)
print("\nAnswer from Bert: ",ans)

Length of the passage: 205 words

Answer from Bert:  england ’ s national summer sport , which is now played throughout the world , particularly in australia , india , pakistan , the west indies , and the british isles


In [15]:
#specify your file here, for question and answers
with open("CricketData.txt") as f:
    passage = f.read()

print(f"Length of the passage: {len(passage.split())} words")
temp = True
print("To Exit from QNA Please enter False ")
while temp == True:
    question1 = input("Enter your question: ")
    if question1 == "False":
        break
    elif len(question1) > 2:
        start_index, end_index, _, _,ans = bert_question_answer(question1,passage)
        print("\nAnswer from Bert: ",ans)
        print("\nStarting Index: ",start_index)
        print("\nEnding Index: ",end_index)
    else:
        print("please Enter proper question")

Length of the passage: 415 words
To Exit from QNA Please enter False 
Enter your question: howmuch wide is pitch


Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.



Answer from Bert:  3 . 04 metres )

Starting Index:  93

Ending Index:  97
Enter your question: what is england nation summer sport


Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.



Answer from Bert:  cricket

Starting Index:  8

Ending Index:  8
Enter your question: False
